In [ ]:
import os
import nivapy3 as nivapy
import pandas as pd

# Download daily river discharge data from NVE

Stations of interest for AquaInfra use case:

* **Numedalslågen**: Holmfoss i Numedalslågen, 15.61.0, from 1970. About 15 km upstream of land-sea outflow.
* **Drammenselva**
    - Mjøndalen bru, 12.534.0, from 2004. Most D/S station
    - Døvikfoss, 12.285.0, from 1912. About 25 km U/S of Mjøndalen bru, no huge tribs between the two. Just downstream of Tyrifjorden and Gravfoss Kraftverk
* **Glomma**
    - Solbergfoss, 2.605.0, from 1964. Just downstream of Øyeren. Was the most downstream station for a long time. Estimated rather than measured? «Data ved Solbergfoss beregnes ut fra kraftverksproduksjon og tapping i luker»
    - Sarpsborg, 2.489.0, from 2009, Most D/S station. Not in Sildre though, only in NVE Atlas. Lets see if we can get it. Otherwise go for Solbergfoss. **Update: Not available in HydAPI, so drop for now.**

## User input

In [ ]:
# Dictionary of stations we want per river of interest
stn_id_name_dict = {
    " Nessedalselv": {"79.3.0 Versjon 1": " Nessedalselv"},

    "Sogndalsvatn": {"77.3.0 Versjon 99": " Sogndalsvatn"},

    " Fornabu": {"74.18.0 Versjon 1": " Fornabu"},

    " Årdalsvatn": {"74.1.0 Versjon 1": " Årdalsvatn"},

    " Krokenelv": {"75.23.0 Versjon 1": " Krokenelv"},

    " Feigumfoss": {"75.28.0 Versjon 1": " Feigumfoss"},

    " Lærdalselv": {"73.130.0 Versjon 3": " Lærdalselv"},

    " Flåm bru": {"72.77.0 Versjon 1": " Flåm bru"},

    " Tistel": {"70.7.0 Versjon 1": " Tistel"},

    " Mel": {"78.12.0 Versjon 1": " Mel"},

    " Bøyumselv": {"78.8.0 Versjon 2": " Bøyumselv"},

    " Ullebøelv": {"80.4.0 Versjon 1": " Ullebøelv"},


}

# Parameters to download
par_ids = [1001]  # Daily mean discharge

# Start and end dates for download
st_dt = "2015-12-31"
end_dt = "2024-01-01"

# Where to save data
download_date = "2025-09-23"  # (for file naming)
out_folder = r'../../data/river/discharge/raw'
metadata_folder = r'../../data/river/discharge'

## Check whether stations are available in HydAPI & save metadata for stations with data

In [ ]:
# List all stations
stn_df = nivapy.da.get_nve_hydapi_stations()
print(len(stn_df), "stations available in HydAPI.")

# Check whether desired stations are available
station_id_li = [stn_id for river in stn_id_name_dict.values() for stn_id in river.keys()]
missing_station_ids = []

for stn_id in station_id_li:
    # Check if the station ID is in the DataFrame's 'station_id' column
    if stn_id not in stn_df['station_id'].values:
        missing_station_ids.append(stn_id)

print(f"Desired stations missing from HydAPI: {missing_station_ids}")

# Drop missing stations from the dictionary of stations we want to download data for
for river, stations in stn_id_name_dict.items():
    for stn_id in list(stations.keys()):  # Use list() to avoid modifying the dictionary while iterating
        if stn_id in missing_station_ids:
            del stations[stn_id]

# Drop missing stations from station_id_li
station_id_li = [stn_id for stn_id in station_id_li if stn_id not in missing_station_ids]

print("New river & station_id dictionary:")
print(stn_id_name_dict)

# Extract metadata for these stations & save to csv
select_stn_df = stn_df[stn_df['station_id'].isin(station_id_li)]
fpath = os.path.join(metadata_folder, "discharge_stations_metadata.csv")
select_stn_df.to_csv(fpath)

## Download data

See here for an interpretation of the correction and quality integer values in the data: https://hydapi.nve.no/UserDocumentation/

Quality types:
* Unknown = 0
* Uncontrolled = 1
* PrimaryControlled = 2
* SecondaryControlled = 3

Correction type indicates whether the value has been corrected. e.g.:
* 0: No changes
* 1: Manual- or ice correction
* 2: Interpolation
* 3: Value computed with models or/and other series
* 4: Daily mean computed with arithmetic mean. (Only used on series where we normally should use discharge curve)
* 13: The value is calculated based on a similar station or a station close by
* 14: The value is missing, and is only calculated based on statistical information for the data series

In [ ]:
# Get daily mean discharge data
# (resolution= 1440; other resolutions not working with HydAPI last time I checked)
for river, station_dict in stn_id_name_dict.items():
    for station_id, station_name in station_dict.items():
        df = nivapy.da.query_nve_hydapi([station_id], par_ids, st_dt, end_dt, resolution=1440)

        df.set_index('datetime', inplace=True)

        # Save to csv
        fname = f"Q_daily-mean_{river}_{station_name}_{station_id}_download-{download_date}.csv"
        df.to_csv(os.path.join(out_folder, fname))
        print(f"Downloaded & saved data for {station_name}, {station_id}")